In [1]:
%reload_ext ishbook
import pandas as pd
import plus
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Pull in the CS Team List and the Agency Domain names

In [38]:
cs_teams = plus.read_google_sheet('1194g28N02sqVJMhHHQthdKItgzgbwWwaAfgC5YVVFko', intype = 'key', sheet = 'Current Team Info')
cs_teams.head(2)

,First Name,Last Name,Common Name,Payroll Name,Team,Role,Manager,Location,AdCentral Name,CS Start Date,...,Username,Indeed.com,Notes/Transition Date (if applicable),Notes Name,Manager Email,Managers' Manager,Level 3 Management,Concatenate,Director,Ad Central Name
,,,,,,,,,,,,,,,,,,,,,
,Aaron,Crawford,Aaron Crawford,Aaron Crawford,CS-EMEA-UK-NationalAccounts,Senior CSS,Elizabeth Kalyana,LON,ACraw,1/18/2016,...,acrawford,indeed.com,,"Crawford, Aaron",elizabethk@indeed.com,sfowler@indeed.com,deepa@indeed.com,"elizabethk@indeed.com,sfowler@indeed.com,deepa...",Diane Melchionne,Aaron Crawford
,Aaron,Doran,Aaron Doran,Aaron Doran,Training,T&D Coordinator,Christine Younkin,DUB,ADora,9/28/2015,...,adoran,indeed.com,,"Doran, Aaron",cyounkin@indeed.com,eileen@indeed.com,,"cyounkin@indeed.com,eileen@indeed.com,",Eileen Bowen,Aaron Doran


In [39]:
email_domains = pd.read_csv("agency_domains.csv")
email_domains.head()

,Unnamed: 0,Email Domains
0,0,mediactive.net
1,1,groupeoptimisation.com
2,2,advendors.com
3,3,hirepower.ca
4,4,ca.crh.com


## Filter the CS Team List so that we are only pulling Agency teams & reps

- The print statements are just checking to make sure I only captured reps from those teams - they can be ignored/depricated

In [11]:
agency_emails_list = ["CSAgenciesSTM2","CSAgenciesSTM1","CSAgenciesSTM3","CSAgenciesNY1","CSAgenciesNY2"]

agency_emails = cs_teams[cs_teams["Team"].isin(agency_emails_list)]

print len(agency_emails)
print
print agency_emails['Team'].value_counts()
agency_emails.tail(3)

45

CSAgenciesSTM2    10
CSAgenciesNY2      9
CSAgenciesSTM1     9
CSAgenciesSTM3     9
CSAgenciesNY1      8
Name: Team, dtype: int64


,First Name,Last Name,Common Name,Payroll Name,Team,Role,Manager,Location,AdCentral Name,CS Start Date,...,Username,Indeed.com,Notes/Transition Date (if applicable),Notes Name,Manager Email,Managers' Manager,Level 3 Management,Concatenate,Director,Ad Central Name
,,,,,,,,,,,,,,,,,,,,,
347,Stephanie,Pianko,Stephanie Pianko,Stephanie Pianko,CSAgenciesNY1,"Regional Director, National Accounts & Ad Agen...",Laken Masterson,NYC,SPian,6/20/2011,...,stephaniep,indeed.com,,"Pianko, Stephanie",laken@indeed.com,alana@indeed.com,,"laken@indeed.com,alana@indeed.com,",Laken Masterson,Stephanie Pianko
348,Tommy,Domanick,Tommy Domanick,Tommy Domanick,CSAgenciesSTM1,Regional Director,Laken Masterson,STM,TDoma,6/20/2011,...,thomas,indeed.com,,"Domanick, Tommy",laken@indeed.com,alana@indeed.com,,"laken@indeed.com,alana@indeed.com,",Laken Masterson,Tommy Domanick
10152,Zack,Massella,Zack Massella,Zachary Massella,CSAgenciesSTM1,Manager,Tommy Domanick,STM,ZMass,6/3/2013,...,zackm,indeed.com,,"Massella, Zack",thomas@indeed.com,laken@indeed.com,alana@indeed.com,"thomas@indeed.com,laken@indeed.com,alana@indee...",Laken Masterson,Zack Massella


## Create a list of rep emails from the filtered CS teams - we'll be using these to feed into the IQL pulls

In [12]:
email_tuple = tuple([str(x) for x in agency_emails.Email])

## From crmemails, we'll need to distinct data sets

- **agency_to_emails** will give us the emails that our Agency reps have received from 

In [44]:
q4_start = 2016-10-01
q4_end = 2016-12-31

In [46]:
%ish agency_to_emails = from crmemails {q4_start} {q4_end} where to_address in {email_tuple} \
group by from_address, to_address \
select counts /*Email Count*/
output = None

In [47]:
agency_to_emails["Status"] = "Inbound"

In [48]:
%ish agency_from_emails = from crmemails {q4_start} {q4_end} where from_address in {email_tuple} \
group by from_address, to_address \
select counts /*Email Count*/
output = None

In [49]:
agency_from_emails["Status"] = "Outbound"

In [50]:
agency_to_emails.head()
agency_from_emails.head(2)

,from_address,to_address,Email Count,Status
0,amanda@indeed.com,"""abate",23,Outbound
1,kristie@indeed.com,"""adams",4,Outbound


## Convert to Strings

In [51]:
agency_to_emails.rename(columns = {"to_address" : "Email 1", "from_address" : "Email 2"}, inplace = True)
agency_to_emails["Email 1"] = agency_to_emails["Email 1"].astype("str")
agency_to_emails["Email 2"] = agency_to_emails["Email 2"].astype("str")

agency_from_emails.rename(columns = {"from_address" : "Email 1", "to_address" : "Email 2"}, inplace = True)
agency_from_emails["Email 1"] = agency_from_emails["Email 1"].astype("str")
agency_from_emails["Email 2"] = agency_from_emails["Email 2"].astype("str")

## Concate Two Frames

In [52]:
combined = pd.concat([agency_to_emails,agency_from_emails], ignore_index = True)

# Get Rid of Emails with Invalid Structure (No @ Symbol)

In [53]:
condition = combined["Email 1"].str.split("@").str.len() > 1
combined = combined[condition]

condition = combined["Email 2"].str.split("@").str.len() > 1
combined = combined[condition]

## Remove Emails from @indeed.com to @indeed.com

In [54]:
condition1 = combined["Email 1"].str.split("@").str.get(1) == "indeed.com"
condition2 = combined["Email 2"].str.split("@").str.get(1) == "indeed.com"

combined = combined[~(condition1 & condition2)]

## Remove Emails Not in Email Domains

In [55]:
condition = combined["Email 2"].str.split("@").str.get(1).isin(email_domains["Email Domains"])

combined[~condition].to_csv("non_agencydomain_email2.csv")
combined = combined[condition]

In [56]:
grouped = combined.groupby(["Email 1", "Email 2", "Status"])
results = grouped["Email Count"].sum().to_frame().unstack().reset_index().fillna(0)
results["Total"] = results[("Email Count", "Inbound")] + results[("Email Count", "Outbound")]
results.head(3)

Email 1                     Email 2 Email Count           \
Status                                                   Inbound Outbound   
0       achin@indeed.com       benjamin.pack@tmp.com         0.0      3.0   
1       achin@indeed.com         marie.smith@adp.com         0.0      1.0   
2       achin@indeed.com  stacy@ramseymediaworks.com         1.0      1.0   

       Total  
Status        
0        3.0  
1        1.0  
2        2.0

In [57]:
results.tail()

Email 1                      Email 2 Email Count           \
Status                                                    Inbound Outbound   
1156    zackm@indeed.com          niki@recruitics.com         4.0      3.0   
1157    zackm@indeed.com  patrick.mitchell@equest.com         0.0      2.0   
1158    zackm@indeed.com        s.walsh@jobtarget.com         0.0      4.0   
1159    zackm@indeed.com           support@equest.com         0.0      1.0   
1160    zackm@indeed.com           tim@recruitics.com         0.0      1.0   

       Total  
Status        
1156     7.0  
1157     2.0  
1158     4.0  
1159     1.0  
1160     1.0

In [58]:
final_send_list = results[results["Email Count"]["Outbound"] >= 3]

In [59]:
final_send_list

Email 1                                 Email 2  \
Status                                                                 
0           achin@indeed.com                   benjamin.pack@tmp.com   
5          alyssa@indeed.com              cbringardner@group1201.com   
6          alyssa@indeed.com                  ccampbell@appvault.com   
7          alyssa@indeed.com                    haley@looklisten.com   
9          alyssa@indeed.com                   jwilkes@group1201.com   
10         alyssa@indeed.com                    kbates@group1201.com   
11         alyssa@indeed.com               kschleicher@group1201.com   
12         alyssa@indeed.com                   ktaylor@group1201.com   
13         alyssa@indeed.com            lilia@patriotadvertising.com   
15         alyssa@indeed.com                  mvasquez@group1201.com   
16         alyssa@indeed.com           pagano@patriotadvertising.com   
18         alyssa@indeed.com             soheyl.shakib@group1201.com   
20         alyssa@indeed.com              tim@patriotadvertising.com   
21         alyssa@indeed.com                   tmiller@group1201.com   
23         amanda@indeed.com             aprentice@publicstorage.com   
25         amanda@indeed.com                  denise.daly@equest.com   
26         amanda@indeed.com              jmasella@publicstorage.com   
28         amanda@indeed.com                       jsmasek@gmail.com   
29         amanda@indeed.com                laura.bellamy@equest.com   
30         amanda@indeed.com              lindsay.selders@equest.com   
31         amanda@indeed.com                lisa.thompson@equest.com   
33         amanda@indeed.com                     mutan.li@equest.com   
34         amanda@indeed.com             patrick.mitchell@equest.com   
37        amandam@indeed.com                        abm3dj@gmail.com   
43        amandam@indeed.com               tish@ramseymediaworks.com   
45       anthonyb@indeed.com             angie@hightowerservices.com   
48       anthonyb@indeed.com         brandiebowden@randallreilly.com   
49       anthonyb@indeed.com  chisha.collins@randstadsourceright.com   
50       anthonyb@indeed.com           cwright@hightowerservices.com   
51       anthonyb@indeed.com                   dbarton@johngroup.com   
...                      ...                                     ...   
1096         samk@indeed.com                      codyh@bayardad.com   
1098         samk@indeed.com                      jackf@bayardad.com   
1099         samk@indeed.com                    josue@ripplemedia.io   
1100         samk@indeed.com                     kevinr@bayardad.com   
1102         samk@indeed.com                   michaelh@bayardad.com   
1105         samk@indeed.com                    stevens@bayardad.com   
1108    scalderon@indeed.com         afunk@transworldadvertising.com   
1109    scalderon@indeed.com         brandiebowden@randallreilly.com   
1110    scalderon@indeed.com         chris@transworldadvertising.com   
1112    scalderon@indeed.com            danajordan@randallreilly.com   
1113    scalderon@indeed.com           danarocheck@randallreilly.com   
1114    scalderon@indeed.com           emilystrong@randallreilly.com   
1115    scalderon@indeed.com       garetttoflinski@randallreilly.com   
1117    scalderon@indeed.com               jtpayne@randallreilly.com   
1118    scalderon@indeed.com       katelynboisvert@randallreilly.com   
1119    scalderon@indeed.com    kendallholderfield@randallreilly.com   
1120    scalderon@indeed.com             kimwalker@randallreilly.com   
1121    scalderon@indeed.com           leewilliams@randallreilly.com   
1122    scalderon@indeed.com         mallorywalker@randallreilly.com   
1123    scalderon@indeed.com           mattschmitt@randallreilly.com   
1124    scalderon@indeed.com          michaeladams@randallreilly.com   
1126    scalderon@indeed.com       nichelecantrell@randallreilly.com   
1131    scalderon@indeed.com                 sarah.palmer@ppr

In [18]:
results.to_csv("check_interactions7.csv")